In [ ]:
# Load libraries
import missingno as msno 
import re
import pandas as pd
import numpy as np


### Load and preprocess the csv files

In [ ]:
# Load Data
#output_1880 = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt6-filter11.csv', index_col=0)
#output_1880 = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/doubt/not so confident7.csv', index_col=0)
#output_1880 = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/mn_1850/doubt-mn1850-v7 new.csv',index_col=0)

output_1880 = pd.read_csv('mn-1880-3-half2-input.csv')

Identify how many records have H address null but a full address present

In [ ]:

print(len(output_1880.loc[(output_1880['H Address'].isnull()) & (output_1880['Full Address'].notnull())]))

0


Identified that in previous file, there were instances where h address was null even though full address wasnt null

Hence repeated the address formatting process as in cd_processing task

In [ ]:
# create a list of cities
cities = ['Hoboken', 'N. J.', 'Spain', 'B’klyn', 'Mt. Vernon','Brooklyn','Jersey City']
def has_city(address, cities = cities):
  for city in cities:
    if city in address:
      return city
  return ''

In [ ]:
# similarly do it for other directories

for i in range(1,len(output_1880)):
  if (str(output_1880["H Address"][i]) =="nan") & (str(output_1880["Full Address"][i]) !="nan"):
    a = output_1880["Full Address"][i].split(",")
    output_1880["H Address"][i] = ' '.join(a[1:])

    address_1 = output_1880["H Address"][i].split()
    if ((address_1[0] == 'h') or (address_1[0] == 'h.')):
        output_1880['H Status-flag'][i] = 'True'
        start = 1
    else:
        output_1880['H Status-flag'][i] = 'Assume'
        start = 0
    #print(address_1)
    house_num = False
    if address_1[start].isdigit():
        house_num = True
    elif (len(address_1) >2):
      if (address_1[start+1].isdigit()):
        house_num = True
        start += 1
    elif (len(address_1) >1):
      if start ==1:
        continue
      elif (address_1[start+1].isdigit()):
        house_num = True
        start += 1
    
    if house_num:
        output_1880['H House Number'][i] = address_1[start]
        #city
        city = has_city(cities, address_1)
        if city:
          output_1880['H City'][i] = city
          output_1880['H Street Name'][i] = ' '.join(address_1[(start+1):]).replace(city,'').strip()
        else:
          output_1880['H Street Name'][i] = ' '.join(address_1[(start+1):])
    else:
        output_1880['H City'][i] = ' '.join(address_1[start:])

    

Save the files as intermediate datsets

In [ ]:
#output_1880.to_csv("data/output_1880_int.csv")
#output_1850_mn.to_csv("data/output_1850_mn.csv")
#output_1850_bk.to_csv("data/output_1850_bk.csv")

### Load cities and street directory data
1. counties, city and street names are imported. Streets are taken from street diectory output
2. combine street names from street directories
3. Add more combinations to street names like numbers reported in words, include AV prefix to certain streets
4. Add differetn variations of cities (like spelling differences)

In [ ]:
# Load cities and street directory data
state_counties = pd.read_csv("us_cities_states_counties.csv", encoding = "ISO-8859-1", engine='python')
uscities = pd.read_csv("uscities.csv", encoding = "ISO-8859-1", engine='python')
bk1850_street_names = pd.read_csv("full_bk_dict.csv", encoding = "ISO-8859-1", engine='python')
mn1850_street_names = pd.read_csv("full_mn_dict.csv", encoding = "ISO-8859-1", engine='python')


In [ ]:
# combine street names from street directories
bk1850_street_names.drop(['ED'], axis=1)
mn1850_street_names.drop(['ED'], axis=1)

bk_street_names = list(pd.unique(bk1850_street_names.values.ravel('K')))
mn_street_names = list(pd.unique(mn1850_street_names.values.ravel('K')))

bk_street_names[2000]

street_names = list(set(bk_street_names))
street_names.extend(mn_street_names)
street_names = list(set(street_names))
street_names = street_names[1:]

# include changes to the street names like:
# add avenue AV, modify street numbers written in words
# WILLIAMSBURG, BUSHWICK, GREENPOINT,  flatbush, 'RAVENSWOOD','CAMPTOWN','CARMANSVLLE', 'WMSBURG','WULIAMSBURG', 'WMBURG','WMSBG','WMSBURGH' can be included later to brooklyn
street_names.extend(['IRVING HOUSE', 'WEST NEW BRIGHTON', 'WYCK', 'WASHGTN','BWRY','HOUSTON','THIRTY-FIRST','FORTYTHIRD','ATLANTIC','EASTRN PEARL'])
street_names.extend(['FIRST','SECOND','THIRD','FOURTH','FIFTH','SIXTH','SEVENTH','EIGHTH','NINTH','TENTH','ELEVENTH', 'TWELFTH','THIRTEENTH','FOURTEENTH'])

street_names.extend(['GREENWICH', 'MADISON', 'LEXINGTON', 'WASHINGTON','B.WAY', 'BRCLY', 'BROADWAV'])

for i in range(len(street_names)):
  if street_names[i] in ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z'):
    street_names[i] = 'AV ' + street_names[i]
print(len(street_names))

street_names = list(set(street_names))

ordinals_list = ['FIRST','SECOND','THIRD','FOURTH','FIFTH','SIXTH','SEVENTH','EIGHTH','NINTH','TENTH','ELEVENTH', 'TWELFTH','THIRTEENTH','FOURTEENTH','FIFTEENTH','SIXTEENTH','SEVENTEENTH','EIGHTEENTH','NINETEENTH','TWENTIETH','TWENTY-FIRST','TWENTY-SECOND','TWENTY-THIRD','TWENTY-FOURTH','TWENTY-FIFTH','TWENTY-SIXTH','TWENTY-SEVENTH','TWENTY-EIGHTH','TWENTY-NINTH','THIRTIETH','THIRTY-FIRST','THIRTY-SECOND','THIRTY-THIRD','THIRTY-FOURTH','THIRTY-FIFTH','THIRTY-SIXTH','THIRTY-SEVENTH','THIRTY-EIGHTH','THIRTY-NINTH','FOURTIETH','FOURTY-FIRST','FOURTY-SECOND','FOURTY-THIRD','FOURTY-FOURTH','FOURTY-FIFTH','FOURTY-SIXTH','FOURTY-SEVENTH','FOURTY-EIGHTH','FOURTY-NINTH','FIFTIETH','FIFTY-FIRST','FIFTY-SECOND','FIFTY-THIRD','FIFTY-FOURTH','FIFTY-FIFTH','FIFTY-SIXTH','FIFTY-SEVENTH','FIFTY-EIGHTH','FIFTY-NINTH','TWENTY8TH','TWENTY3RD','TWENTY .7TH','TWENTY7TH','TWENTY5TH']

replace_list = ['1ST','2ND','3RD','4TH','5TH','6TH','7TH','8TH','9TH','10TH','11TH','12TH','13TH','14TH','15TH','16TH','17TH','18TH','19TH','20TH','21ST','22ND','23RD','24TH','25TH','26TH','27TH','28TH','29TH','30TH','31ST','32ND','33RD','34TH','35TH','36TH','37TH','38TH','39TH','40TH','41ST','42ND','43RD','44TH','45TH','46TH','47TH','48TH','49TH','50TH','51ST','52ND','53RD','54TH','55TH','56TH','57TH','58TH','59TH','28TH','23RD','27TH','27TH','25TH']

4452


In [ ]:
state_counties = state_counties.loc[(state_counties['State full'] == 'New York') | (state_counties['State full'] == 'New Jersey') | (state_counties['State full'] == 'Connecticut')]
state_counties['City'] = state_counties['City'].str.upper()
state_counties['County'] = state_counties['County'].str.upper()
state_counties['City alias'] = state_counties['City alias'].str.upper()
state_counties.head()

,City,State short,State full,County,City alias
0,HOLTSVILLE,NY,New York,SUFFOLK,INTERNAL REVENUE SERVICE
1,HOLTSVILLE,NY,New York,SUFFOLK,HOLTSVILLE
6097,AVON,CT,Connecticut,HARTFORD,AVON
6098,BLOOMFIELD,CT,Connecticut,HARTFORD,BLOOMFIELD
6099,WINDSOR,CT,Connecticut,HARTFORD,WINDSOR


In [ ]:
uscities = uscities.loc[(uscities['state_name'] =="New York") | (uscities['state_name'] =='New Jersey') | (uscities['state_name'] == 'Connecticut')]
uscities['city'] = uscities['city'].str.upper()
uscities['county_name'] = uscities['county_name'].str.upper()
uscities.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
3171,CARLSTADT,Carlstadt,NJ,New Jersey,34003,BERGEN,34003,Bergen,40.8247,-74.0613,6314.0,617.0,polygon,False,True,America/New_York,3,07072,1840000895
3172,UPPER POHATCONG,Upper Pohatcong,NJ,New Jersey,34041,WARREN,34041,Warren,40.6774,-75.1558,1586.0,947.0,polygon,False,True,America/New_York,3,08865,1840039970
3173,ZAREPHATH,Zarephath,NJ,New Jersey,34035,SOMERSET,34035,Somerset,40.5361,-74.5745,9.0,8.0,polygon,False,False,America/New_York,3,08873 08890,1840024282
3174,TRENTON,Trenton,NJ,New Jersey,34021,MERCER,34021,Mercer,40.2236,-74.7641,296869.0,4328.0,polygon,False,True,America/New_York,2,08638 08611 08618 08629 08609 08608 08601 0860...,1840001383
3175,NORTH CAPE MAY,North Cape May,NJ,New Jersey,34009,CAPE MAY,34009,Cape May,38.9765,-74.9516,2772.0,772.0,polygon,False,False,America/New_York,3,08204,1840005866


In [ ]:
# cities list 
cities = []
cities.extend(uscities['city'])
cities.extend(state_counties['City'])
cities.extend(state_counties['City alias'])
cities = list(set(cities))
cities = [e for e in cities if e not in street_names]
print(len(cities))

imp_cities = []
imp_cities.extend(['WILLIAMSBURG', 'BUSHWICK', 'GREENPOINT',  'FLATBUSH', 'RAVENSWOOD','CAMPTOWN','CARMANSVLLE', 'WMSBURG','WULIAMSBURG', 'WMBURG','WMSBG','WMSBURGH', 'HOBOKEN', 'N. J.', 'S. ISLAND','SPAIN', 'B’KLYN', 'MT. VERNON','BROOKLYN','JERSEY CITY', 'N. JERSEY', 'CINCINNATI', 'GERMANY', 'STNTN', 'JERSEY C.', 'HOBOK', 'CONNECTICUT','BLYN',"JERSY CTY",'BRKLYN','NEW JERSEY','LONG ISLAND', 'N. ORLEANS', 'BKLYN'])
imp_cities.extend(['BROOKLYR','B.KLYN','JER. C.','CONN', 'L. I.', 'L.I.', 'L. I','L.I','L I','L I.', 'PTCHESTER', "B'KLYN",'J. C.','N.J.','S. I.','S.I.', 'NEW J'])


# county list
counties = []
counties.extend(uscities['county_name'])
counties.extend(state_counties['County'])
counties = list(set(counties))
print(len(counties))

4999
89


### functions to identify city, county and street

In [ ]:

# functions to find city, county, streets
def has_city(address, cities = cities):
  string_match = []
  for city in cities:
    if city in address:
      string_match.append(city)
  string_match.append('')
  return max(string_match, key = len) 


def has_impcity(address, cities = imp_cities):
  for city in cities:
    if city in address:
      return city
  return ''


def has_county(address, counties = counties):
    string_match = []
    for county in counties:
      if county in address:
        string_match.append(county)
    string_match.append('')
    return max(string_match, key = len) 


def has_street(address, streetnames = street_names, city = ''):
    street_match = []
    street_match_1 = []
    street_match_2 = []
    street_match.append('')
    street_match_1.append('')
    street_match_2.append('')
    address_1 = ''
    address_2 = ''
    special_cases = ''

    # Handle special cases like corner and near
    if 'C.' in address:
      address_1 = address[:address.index("C.")] # divide the record by C
      address_2 = address[address.index("C.") +1 :]
      special_cases = 'corner'
      
    elif 'N.' in address:
      address_1 = address[:address.index("N.")] # divide the record by C
      address_2 = address[address.index("N.") +1 :]
      #print(address_1)  
      special_cases = 'near'
    
    address_1 = address_1.replace('.','').strip()
    address_2 = address_2.replace('.','').strip()
    address = address.replace('.','').strip()
    
    # find streets from overall and split record
    for street in streetnames:
      if str(street) in address_1:
        street_match_1.append(str(street))
      if str(street) in address_2:
        street_match_2.append(str(street))
      if str(street) in address:
        street_match.append(str(street))

    street_1 = max(street_match_1, key = len) 
    street_2 = max(street_match_2, key = len) 
    street = max(street_match, key = len)
    
    # if both street 1 and street 2 exist then it qualifies for corner or near special case
    if (street_1 != '') & (street_2 != ''):
      street =  ''
    if (street_1 == '') | (street_2 == ''):
      street_1 = ''
      street_2 = ''
      special_cases = ''

   
    return (street, street_1, street_2, special_cases)


In [ ]:
import re
#[re.sub(r'E\.\s?','East ', str(x)) for x in output_1850_mn['H Address']]

output_1880['H Address'] = output_1880['H Address'].str.replace('-','')

In [ ]:
# convert to upper case for consistency
output_1880['H Address'] = output_1880['H Address'].str.upper()

output_1880['H Street Name'] = output_1880['H Street Name'].str.upper()


output_1880['H City'] = output_1880['H City'].str.upper()


### code to modify house nos, streets and city
1. for records starting with H 
2. for  records that have an H tag somewhere in the record

In [ ]:
# code to modify house nos, streets and city

def address(filename):
  filename['new h address-flag'] = ''
  filename['new h house number'] = ''
  filename['new h street name'] = ''
  filename['new h street 1'] = ''
  filename['new h street 2'] = ''
  filename['new h street type'] = ''
  filename['new h city'] = ''
  filename['new h county'] = ''
  filename['unidentified h info'] = ''
  filename['h house number modifier'] = ''
  filename['new address2 house number'] = ''
  filename['new address2 street name'] = ''
  filename['new address2 street 1'] = ''
  filename['new address2 street 2'] = ''
  filename['new address2 street type'] = ''
  filename['new address2 city'] = ''
  filename['new address2 county'] = ''
  filename['unidentified address2 info'] = ''
  filename['address2 house number modifier'] = ''


  count = 1
  #for i in range(272535, len(filename)):
  for i in range(1,len(filename)):
  #for i in range(61167,61174):
    #print(i)
    
    for k in range(len(ordinals_list)): # change street numbers that were written in words
      if ordinals_list[k] in str(filename["H Address"][i]):
        filename["H Address"][i] = str(filename["H Address"][i]).replace(ordinals_list[k], replace_list[k])

    address_1 = str(filename["H Address"][i]).strip().split()
    address_2 = str(filename["H Address"][i]).strip()

    try:
      
      if ((address_1[0] == 'H') or (address_1[0] == 'H.')): # find if an address start with H

        #print(filename["H Street Name"][i])
        filename['new h address-flag'][i] = 'start with h'
        
        city = has_impcity(address_2, imp_cities) # find if there is a city
        if city == '':
          city = has_city(address_2, cities)
        if city :
          address_2 = address_2.replace(str(city),'').strip() 
          address_1 = address_2.split()
          filename['new h city'][i] = city # if city save it


        house_num = False

        for j in range(len(address_1)):
          if '.5' in address_1[j]: # remove .5 to correctly identify house nos
            filename['h house number modifier'][i] = address_1[j]
            address_1[j] = address_1[j].replace('.5', '').strip()
            
        
        street = ''
        street_1 = ''
        street_2 = ''
        special_cases = ''
        if (len(address_1) ==2 ): # address length is 2
            street, street_1, street_2, special_cases = has_street(str(address_1[1])) # no house no
          
        if (len(address_1) >2): # address len more than 2
            if (address_1[1].isdigit()): # first find a house no
              street, street_1, street_2, special_cases = has_street(' '.join(address_1[2:]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 1
            elif (address_1[2].isdigit()):
              street, street_1, street_2, special_cases = has_street(' '.join(i for i in address_1[1:] if i != address_1[2]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 2
            
            if house_num == False: # there is a street but no house no
                street, street_1, street_2, special_cases = has_street(address_2[1:])
            
        if (street != '') | (special_cases != ''): # for consistency in data remove . from street identifiers
            address_2 = address_2.replace('W.','W').strip()
            address_2 = address_2.replace('E.','E').strip()
            address_2 = address_2.replace('ST.','ST').strip()
            address_2 = address_2.replace('AV.','AV').strip()
            address_2 = address_2.replace('A.V.','AV').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street_1),'').strip()
            address_2 = address_2.replace(str(street_2),'').strip()
            
            filename['new h street name'][i] = street
            filename['new h street 1'][i] = street_1
            filename['new h street 2'][i] = street_2
            filename['new h street type'][i] = special_cases
    
        if house_num:
            filename['new h house number'][i] = address_1[start]
            address_2 = address_2.replace(address_1[start],'').strip()
        
        # find a county
        county = has_county(address_2, counties)
        if county:
          address_2 = address_2.replace(str(county),'').strip()
          filename['new h county'][i] = county
        
        address_2 = address_2.replace('H ','').strip()
        address_2 = address_2.replace('H.','').strip()
        address_2 = address_2.replace('.5','').strip()
        filename['unidentified h info'][i] = address_2 # store the remaining information as unidentified
    
    except:
      pass
    
        
    if count % 1000 == 0:
      print(count)
      #break
    count = count + 1
  

      

  return filename
    

In [ ]:
output_1880_int = address(output_1880)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to b

1000
2000
3000
4000
5000
6000
7000


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
output_1880_int['new h address-flag'].unique()

array(['', 'start with h'], dtype=object)

Save Intermediate files again

In [ ]:
#output_1850_bk_int.to_csv("data/output_1850_bk_int.csv")

In [ ]:
#output_1880_int.to_csv("data/output_1880_int.csv")

In [ ]:
#output_1850_mn_int.to_csv("data/output_1850_mn_int.csv")

Repeat the process for records that have both work and house address

In [ ]:
# repeat the above process for records that have both work and house address

def h_inbw_address(filename):
  count = 1
  #for i in range(272535,len(filename)):
  for i in range(1,len(filename)):
  #for i in range(61167,61700):

    try:
    
      address_3 = str(filename["H Address"][i]).strip().split()
      
      if (address_3[0] != 'H') & (address_3[0] != 'H.') & ('H' in address_3[1:]):
        #print(address_1.index("H"))
        #print(filename["H Address"][i])

        filename['new h address-flag'][i] = 'h in between'

        address_2 = ' '.join(address_3[address_3.index("H"):])
        #print(address_2)
        #print(filename['new h address-flag'][i])
              
        city = has_impcity(address_2, imp_cities) # find if there is a city
        if city == '':
          city = has_city(address_2, cities)
        if city :
          address_2 = address_2.replace(str(city),'').strip() 
          filename['new h city'][i] = city # if city save it

        address_1 = address_2.split()

        house_num = False

        for j in range(len(address_1)):
          if '.5' in address_1[j]:
            filename['h house number modifier'][i] = address_1[j]
            address_1[j] = address_1[j].replace('.5', '').strip()

        street = ''
        street_1 = ''
        street_2 = ''
        special_cases = ''
        if (len(address_1) ==2 ): # length is 2
            street, street_1, street_2, special_cases = has_street(str(address_1[1])) # no house no
          
        if (len(address_1) >2): # len more than 2
            if (address_1[1].isdigit()): 
              street, street_1, street_2, special_cases = has_street(' '.join(address_1[2:]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 1
            elif (address_1[2].isdigit()):            
              street, street_1, street_2, special_cases = has_street(' '.join(i for i in address_1[1:] if i != address_1[2]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 2
            
            if house_num == False: # there is a street but no house no
                street, street_1, street_2, special_cases = has_street(address_2[1:])
            
        if (street != '') | (special_cases != ''):
            address_2 = address_2.replace('W.','W').strip()
            address_2 = address_2.replace('E.','E').strip()
            address_2 = address_2.replace('ST.','ST').strip()
            address_2 = address_2.replace('AV.','AV').strip()
            address_2 = address_2.replace('A.V.','AV').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street_1),'').strip()
            address_2 = address_2.replace(str(street_2),'').strip()
            
            filename['new h street name'][i] = street
            filename['new h street 1'][i] = street_1
            filename['new h street 2'][i] = street_2
            filename['new h street type'][i] = special_cases
    
        if house_num:
            filename['new h house number'][i] = address_1[start]
            address_2 = address_2.replace(address_1[start],'').strip()
          
        
        county = has_county(address_2, counties)
        if county:
          address_2 = address_2.replace(str(county),'').strip()
          filename['new h county'][i] = county
        
        address_2 = address_2.replace('H ','').strip()
        #address_2 = address_2.replace('H.','').strip()
        address_2 = address_2.replace('.5','').strip()
        filename['unidentified h info'][i] = address_2


  # get sec address
        address_2 = ' '.join(address_3[:address_3.index("H")])
        #print(address_2)
        address_1 = address_3[:address_3.index("H")]

        city = has_impcity(address_2, imp_cities) # find if there is a city
        if city == '':
          city = has_city(address_2, cities)
        if city :
          address_2 = address_2.replace(str(city),'').strip() 
          filename['new address2 city'][i] = city # if city save it
        
        address_1 = address_2.split()

        house_num = False
        for j in range(len(address_1)):
          if '.5' in address_1[j]:
            filename['address2 house number modifier'][i] = address_1[j]
            address_1[j] = address_1[j].replace('.5', '').strip()
        
        street = ''
        street_1 = ''
        street_2 = ''
        special_cases = ''
        if (len(address_1) ==1 ): # length is 1
            street, street_1, street_2, special_cases = has_street(str(address_1[0])) # no house no        
        if (len(address_1) >1): # len more than 2
            if (address_1[0].isdigit()): 
              street, street_1, street_2, special_cases = has_street(' '.join(address_1[1:]))
              #print(street, street_1, street_2, special_cases)
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 0
            elif (address_1[1].isdigit()):            
              street, street_1, street_2, special_cases = has_street(' '.join(i for i in address_1 if i != address_1[1]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 1
            
            if house_num == False: # there is a street but no house no
                street, street_1, street_2, special_cases = has_street(address_2)
            
        if (street != '') | (special_cases != ''):
            address_2 = address_2.replace('W.','W').strip()
            address_2 = address_2.replace('E.','E').strip()
            address_2 = address_2.replace('ST.','ST').strip()
            address_2 = address_2.replace('AV.','AV').strip()
            address_2 = address_2.replace('A.V.','AV').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street_1),'').strip()
            address_2 = address_2.replace(str(street_2),'').strip()

            filename['new address2 street name'][i] = street
            filename['new address2 street 1'][i] = street_1
            filename['new address2 street 2'][i] = street_2
            filename['new address2 street type'][i] = special_cases
    
        if house_num:
            filename['new address2 house number'][i] = address_1[start]
            #print(filename['new address2 house number'][i])
            address_2 = address_2.replace(address_1[start],'').strip()
          


        county = has_county(address_2, counties)
        if county:
          address_2 = address_2.replace(str(county),'').strip()
          filename['new address2 county'][i] = county
        
        #address_2 = address_2.replace('H ','').strip()
        #address_2 = address_2.replace('H.','').strip()
        address_2 = address_2.replace('.5','').strip()
        filename['unidentified address2 info'][i] = address_2


      #if count % 1000 == 0:
      #  print(count)

      count = count+1    
    
    except:
      pass

  return filename
    

In [ ]:
output_1880_int_2 = h_inbw_address(output_1880_int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: SettingWithCopyWarning: 
A value is trying to b

In [ ]:
output_1880_int_2['new h address-flag'].unique()

array(['', 'start with h', 'h in between'], dtype=object)

In [ ]:
def h_inbw_address(filename):

  count = 1
  #for i in range(272535,len(filename)):
  for i in range(1,len(filename)):

    try:
    
      address_3 = str(filename["H Address"][i]).strip().split()
      
      if (address_3[0] != 'H') & (address_3[0] != 'H.') & ('H.' in address_3[1:]):
        #print(address_1.index("H"))
        #print(filename["H Address"][i])

        filename['new h address-flag'][i] = 'h in between'

        address_2 = ' '.join(address_3[address_3.index("H."):])
        #print(address_2)
        #print(filename['new h address-flag'][i])
        address_1 = address_3[address_3.index("H."):]
        
        city = has_impcity(address_2, imp_cities) # find if there is a city
        if city == '':
          city = has_city(address_2, cities)
        if city :
          address_2 = address_2.replace(str(city),'').strip() 
          filename['new h city'][i] = city # if city save it

        address_1 = address_2.split()

        house_num = False

        for j in range(len(address_1)):
          if '.5' in address_1[j]:
            filename['h house number modifier'][i] = address_1[j]
            address_1[j] = address_1[j].replace('.5', '').strip()

        street = ''
        street_1 = ''
        street_2 = ''
        special_cases = ''
        if (len(address_1) ==2 ): # length is 2
            street, street_1, street_2, special_cases = has_street(str(address_1[1])) # no house no
          
        if (len(address_1) >2): # len more than 2
            if (address_1[1].isdigit()): 
              street, street_1, street_2, special_cases = has_street(' '.join(address_1[2:]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 1
            elif (address_1[2].isdigit()):            
              street, street_1, street_2, special_cases = has_street(' '.join(i for i in address_1[1:] if i != address_1[2]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 2
            
            if house_num == False: # there is a street but no house no
                street, street_1, street_2, special_cases = has_street(address_2[1:])
            
        if (street != '') | (special_cases != ''):
            address_2 = address_2.replace('W.','W').strip()
            address_2 = address_2.replace('E.','E').strip()
            address_2 = address_2.replace('ST.','ST').strip()
            address_2 = address_2.replace('AV.','AV').strip()
            address_2 = address_2.replace('A.V.','AV').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street_1),'').strip()
            address_2 = address_2.replace(str(street_2),'').strip()
            
            filename['new h street name'][i] = street
            filename['new h street 1'][i] = street_1
            filename['new h street 2'][i] = street_2
            filename['new h street type'][i] = special_cases
    
        if house_num:
            filename['new h house number'][i] = address_1[start]
            address_2 = address_2.replace(address_1[start],'').strip()
          
        
        county = has_county(address_2, counties)
        if county:
          address_2 = address_2.replace(str(county),'').strip()
          filename['new h county'][i] = county

        #address_2 = address_2.replace('H ','').strip()
        address_2 = address_2.replace('H.','').strip()
        address_2 = address_2.replace('.5','').strip()
        filename['unidentified h info'][i] = address_2


  # get sec address
        address_2 = ' '.join(address_3[:address_3.index("H.")])
        #print(address_2)
        address_1 = address_3[:address_3.index("H.")]

        city = has_impcity(address_2, imp_cities) # find if there is a city
        if city == '':
          city = has_city(address_2, cities)
        if city :
          address_2 = address_2.replace(str(city),'').strip() 
          filename['new address2 city'][i] = city # if city save it

        address_1 = address_2.split()
        
        house_num = False
        for j in range(len(address_1)):
          if '.5' in address_1[j]:
            filename['address2 house number modifier'][i] = address_1[j]
            address_1[j] = address_1[j].replace('.5', '').strip()
        
        street = ''
        street_1 = ''
        street_2 = ''
        special_cases = ''
        if (len(address_1) ==1 ): # length is 1
            street, street_1, street_2, special_cases = has_street(str(address_1[0])) # no house no        
        if (len(address_1) >1): # len more than 2
            if (address_1[0].isdigit()): 
              street, street_1, street_2, special_cases = has_street(' '.join(address_1[1:]))
              #print(street, street_1, street_2, special_cases)
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 0
            elif (address_1[1].isdigit()):            
              street, street_1, street_2, special_cases = has_street(' '.join(i for i in address_1 if i != address_1[1]))
              if (street != '') | (special_cases != ''): # there is a house no and a street name
                house_num = True
                start = 1
            
            if house_num == False: # there is a street but no house no
                street, street_1, street_2, special_cases = has_street(address_2)
            
        if (street != '') | (special_cases != ''):
            address_2 = address_2.replace('W.','W').strip()
            address_2 = address_2.replace('E.','E').strip()
            address_2 = address_2.replace('ST.','ST').strip()
            address_2 = address_2.replace('AV.','AV').strip()
            address_2 = address_2.replace('A.V.','AV').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street),'').strip()
            address_2 = address_2.replace(str(street_1),'').strip()
            address_2 = address_2.replace(str(street_2),'').strip()

            filename['new address2 street name'][i] = street
            filename['new address2 street 1'][i] = street_1
            filename['new address2 street 2'][i] = street_2
            filename['new address2 street type'][i] = special_cases
    
        if house_num:
            filename['new address2 house number'][i] = address_1[start]
            #print(filename['new address2 house number'][i])
            address_2 = address_2.replace(address_1[start],'').strip()
          
        county = has_county(address_2, counties)
        if county:
          address_2 = address_2.replace(str(county),'').strip()
          filename['new address2 county'][i] = county
        
        #address_2 = address_2.replace('H ','').strip()
        #address_2 = address_2.replace('H.','').strip()
        address_2 = address_2.replace('.5','').strip()
        filename['unidentified address2 info'][i] = address_2  
      
    except:
      pass

  return filename
    

In [ ]:
output_1880_int_2 = h_inbw_address(output_1880_int_2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:145: Sett

In [ ]:
output_1880_int_2['new h address-flag'].unique()

array(['', 'start with h', 'h in between'], dtype=object)

In [ ]:
output_1880_int_2.loc[output_1880_int_2['new h address-flag'] == 'h in between']

,Unnamed: 0,Address 2,Address 2 City,Address 2 House Number,Address 2 Street Name,First Name,Full Address,Full Name,H Address,H City,H House Number,H Status-flag,H Street Name,Last Name,Middle Name,Occupation,Title,Widow Of,Widow-flag,new h address-flag,new h house number,new h street name,new h street 1,new h street 2,new h street type,new h city,new h county,unidentified h info,h house number modifier,new address2 house number,new address2 street name,new address2 street 1,new address2 street 2,new address2 street type,new address2 city,new address2 county,unidentified address2 info,address2 house number modifier
12,272548,NaN,NaN,NaN,NaN,NaN,493 Eighth av. h 323 W,NaN,493 8TH AV. H 323 W,NaN,493.0,Assume,EIGHTH AV. H 323 W,NaN,NaN,NaN,NaN,NaN,False,h in between,,323,,,,,,W,,493,8,,,,,,TH AV,
20,272556,NaN,NaN,NaN,NaN,NaN,Fackler David M. (Rev.) h 216 W 25th,NaN,FACKLER DAVID M. (REV.) H 216 W 25TH,FACKLER DAVID M. (REV.) H 216 W 25TH,NaN,Assume,NaN,NaN,NaN,NaN,NaN,NaN,False,h in between,216,W 25,,,,,,TH,,,,,,,,,FACKLER DAVID M. (REV.),
41,272577,NaN,NaN,NaN,NaN,NaN,Everett William (Rev.) h 44 Second av.,NaN,EVERETT WILLIAM (REV.) H 44 2ND AV.,EVERETT WILLIAM (REV.) H 44 SECOND AV.,NaN,Assume,NaN,NaN,NaN,NaN,NaN,NaN,False,h in between,44,2,,,,,,ND AV,,,WILLIAM,,,,,,EVERETT (REV.),
51,272587,NaN,NaN,NaN,NaN,NaN,315 Fourth av. h 398 Third av.,NaN,315 4TH AV. H 398 3RD AV.,NaN,315.0,Assume,FOURTH AV. H 398 THIRD AV.,NaN,NaN,NaN,NaN,NaN,False,h in between,398,3,,,,,,98 RD AV,,315,4,,,,,,TH AV,
55,272591,NaN,NaN,NaN,NaN,NaN,Everson Charles S. (Rev.) h 56 Monroe,NaN,EVERSON CHARLES S. (REV.) H 56 MONROE,EVERSON CHARLES S. (REV.) H 56 MONROE,NaN,Assume,NaN,NaN,NaN,NaN,NaN,NaN,False,h in between,56,MONROE,,,,,,H,,,CHARLES,,,,,,EVERSON S. (REV.),
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17397,289933,NaN,NaN,NaN,NaN,NaN,1114 Third av. h 314 E 73d,NaN,1114 3RD AV. H 314 E 73D,NaN,1114.0,Assume,THIRD AV. H 314 E 73D,NaN,NaN,NaN,NaN,NaN,False,h in between,314,E 73,,,,,,D,,1114,3,,,,,,RD AV,
17401,289937,NaN,NaN,NaN,NaN,NaN,Abrahams Anshel (Rev.) h 143 E B‚Äôway,NaN,ABRAHAMS ANSHEL (REV.) H 143 E B‚ÄÔWAY,ABRAHAMS ANSHEL (REV.) H 143 E B‚ÄÔWAY,NaN,Assume,NaN,NaN,NaN,NaN,NaN,NaN,False,h in between,,143,,,,,,E B‚ÄÔWAY,,,,,,,,,ABRAHAMS ANSHEL (REV.),
17413,289949,NaN,NaN,NaN,NaN,NaN,29 Maiden la. h 61 E 180th,NaN,29 MAIDEN LA. H 61 E 180TH,NaN,29.0,Assume,MAIDEN LA. H 61 E 180TH,NaN,NaN,NaN,NaN,NaN,False,h in between,61,E 18,,,,,,0TH,,29,MAIDEN LA,,,,,,.,
17434,289970,NaN,NaN,NaN,NaN,NaN,514 First av. h 308 E 74th,NaN,514 1ST AV. H 308 E 74TH,NaN,514.0,Assume,FIRST AV. H 308 E 74TH,NaN,NaN,NaN,NaN,NaN,False,h in between,308,E 74,,,,,,TH,,514,1,,,,,,54 ST AV,


Save the final files

In [ ]:
output_1880_int_2.columns


Index(['Unnamed: 0', 'Address 2', 'Address 2 City', 'Address 2 House Number',
       'Address 2 Street Name', 'First Name', 'Full Address', 'Full Name',
       'H Address', 'H City', 'H House Number', 'H Status-flag',
       'H Street Name', 'Last Name', 'Middle Name', 'Occupation', 'Title',
       'Widow Of', 'Widow-flag', 'new h address-flag', 'new h house number',
       'new h street name', 'new h street 1', 'new h street 2',
       'new h street type', 'new h city', 'new h county',
       'unidentified h info', 'h house number modifier',
       'new address2 house number', 'new address2 street name',
       'new address2 street 1', 'new address2 street 2',
       'new address2 street type', 'new address2 city', 'new address2 county',
       'unidentified address2 info', 'address2 house number modifier'],
      dtype='object')

In [ ]:
#output_1850_mn_int_2.to_csv("data/output_1850_mn_add_std.csv")


In [ ]:
#output_1850_bk_int_2.to_csv("data/output_1850_bk_add_std.csv")


In [ ]:
#output_1880_int_2.to_csv("/Users/prajwal/Desktop/Columbia/C4SR/doubt/doubt6-filter11-s.csv")
#output_1880_int_2.to_csv("/Users/prajwal/Desktop/Columbia/C4SR/doubt/not so confident7-s2.csv")
#output_1880_int_2.to_csv("/Users/prajwal/Desktop/Columbia/C4SR/mn_1850/doubt-mn1850-v7-s-new.csv")

output_1880_int_2.to_csv('mn-1880-3-s-half2.csv')

In [ ]:
output_1880_int_2.head()

,Unnamed: 0,Address 2,Address 2 City,Address 2 House Number,Address 2 Street Name,First Name,Full Address,Full Name,H Address,H City,H House Number,H Status-flag,H Street Name,Last Name,Middle Name,Occupation,Title,Widow Of,Widow-flag,new h address-flag,new h house number,new h street name,new h street 1,new h street 2,new h street type,new h city,new h county,unidentified h info,h house number modifier,new address2 house number,new address2 street name,new address2 street 1,new address2 street 2,new address2 street type,new address2 city,new address2 county,unidentified address2 info,address2 house number modifier
0,272536,NaN,NaN,NaN,NaN,NaN,h 418 E 47th,NaN,H 418 E 47TH,NaN,418.0,TRUE,E 47TH,NaN,NaN,NaN,NaN,NaN,False,,,,,,,,,,,,,,,,,,,
1,272537,NaN,NaN,NaN,NaN,NaN,h 316 E 45th,NaN,H 316 E 45TH,NaN,316.0,TRUE,E 45TH,NaN,NaN,NaN,NaN,NaN,False,start with h,316,E 45,,,,,,TH,,,,,,,,,,
2,272538,NaN,NaN,NaN,NaN,NaN,Bklyn,NaN,BKLYN,NaN,NaN,Assume,NaN,NaN,NaN,NaN,NaN,NaN,False,,,,,,,,,,,,,,,,,,,
3,272539,h 46 Eight,NaN,46.0,Eight,NaN,"h 46 Eight,h. av. Pagan William J,h 746 Tenth av.",NaN,"H. AV. PAGAN WILLIAM J,H 746 10TH AV.","AV. PAGAN WILLIAM J,H 746 TENTH AV.",NaN,TRUE,NaN,NaN,NaN,NaN,NaN,NaN,False,start with h,,N WILLIAM,,,,,,"AV PAGA J,746 10TAV",,,,,,,,,,
4,272540,NaN,NaN,NaN,NaN,NaN,106 W 22d,NaN,106 W 22D,NaN,106.0,Assume,W 22D,NaN,NaN,NaN,NaN,NaN,False,,,,,,,,,,,,,,,,,,,
